In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Описание датасета CrossNER

**Источник:** [Статья (PDF)](https://arxiv.org/pdf/2012.04373)

Датасет **CrossNER** разработан для оценки и изучения способности моделей распознавания именованных сущностей (NER) к обобщению между различными доменами. Он содержит данные из шести областей (доменных подсекций), извлечённые из английской Википедии, а также включает классический набор данных CoNLL2003:

- **AI**: Данные о темах, связанных с искусственным интеллектом.
- **Literature**: Тексты о литературных произведениях, писателях, жанрах.
- **Music**: Материалы о музыке, исполнителях, композициях, жанрах.
- **Politics**: Информация о политиках, политических событиях, партиях.
- **Science**: Данные о научных концептах, учёных, открытиях.
- **CoNLL2003**: Классический эталонный датасет для NER для сравнения.

Каждая доменная подсекция содержит свои специфические типы именованных сущностей, что позволяет оценить способность моделей обобщать знания, полученные в одном домене, на другие.

Структура и особенности датасета:
- Доступны три сплита для каждой доменной подсекции: `train`, `validation`, `test`.
- Набор сущностей и их распределение сильно варьируются от домена к домену.
- Датасет был создан для исследования междоменного переноса и устойчивости NER-моделей. В частности: CoNLL2003 рассматривался как source dataset, a остальные 5 доменов, как target dataset.

![image.png](images/1.png)

**Здесь будет изображение с таблицей статистических характеристик датасета (количество предложений, токенов, сущностей для каждого домена)**

Применение:
- Оценка качества и устойчивости моделей NER при переходе между доменами.
- Изучение влияния доменных различий на результаты распознавания сущностей.
- Разработка более универсальных и обобщающих NER-моделей, способных работать в условиях значительных доменных сдвигов.

ВАЖНО! Так как некоторая оценка и исследование проведено в статье оригинального датасета и их легко "сравнить", давайте попробуем сделать так, чтобы представленные методы метчились в оценками, приведенными в статье. У нас не стоит задача превзойти метрики, представленные там, поскольку нам важно не происследовать перенос с одного домена на другой, а обучить модель, способную на всем представленном датасете (общем) получить похожие метрики и необязательно презвойти их (иначе это не тянет на 2х недельную лабу).

ВАЖНО 2: так как наши одногруппники взяли в рассмотрение одиночный датасет **CoNLL2003**, а он содержится в наших данных мы его не будем выкидывать, но основная наша цель - работать и получать метрики по другим (target) датасетам, однако мы на всякий случай решили оставить **CoNLL2003**, на случай, если содержание этого датасета в наших расширенных данных не вызовет проблем.

# EDA

## Загрузка данных

In [ ]:
from datasets import load_dataset, concatenate_datasets

subsections = ["ai", "conll2003", "literature", "music", "politics", "science"]

train_datasets = []
val_datasets = []
test_datasets = []

for subsection in subsections:
    ds = load_dataset("DFKI-SLT/cross_ner", subsection)
    # Добавляем колонку "source"
    ds_train = ds["train"].map(lambda x: {"source": subsection})
    ds_val = ds["validation"].map(lambda x: {"source": subsection})
    ds_test = ds["test"].map(lambda x: {"source": subsection})
    
    # Собираем в списки
    train_datasets.append(ds_train)
    val_datasets.append(ds_val)
    test_datasets.append(ds_test)

# Объединяем все в единые датасеты
train_dataset = concatenate_datasets(train_datasets)
val_dataset = concatenate_datasets(val_datasets)
test_dataset = concatenate_datasets(test_datasets)



Теперь у нас есть три датасета: train_dataset, val_dataset, test_dataset.
Каждый пример содержит колонку "source" с указанием исходной подсекции.

In [6]:
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'source'],
    num_rows: 14741
})

## Как выглядят данные?

Рассмотрим первый элемент данных, мы видим, что для каждого элемента данных мы имеем:
- индекс конкретного элемента, 
- токены, которые представляют собой каждое отдельно слово
- уже закодированные теги каждого слов
- источник или домен данных, чтобы понимать, из какого датасета данный конкретный пример

In [8]:
import pandas as pd
pd.DataFrame(train_dataset[0])

,id,tokens,ner_tags,source
0,0,Popular,0,ai
1,0,approaches,0,ai
2,0,of,0,ai
3,0,opinion-based,59,ai
4,0,recommender,60,ai
...,...,...,...,...
59,0,5,0,ai
60,0,),0,ai
61,0,:,0,ai
62,0,e12957,0,ai


## Анализ для тегов

Давайте поближе посмотрим на кодировку:

### Что это?

In [15]:
tag2id = {"O": 0, "B-academicjournal": 1, "I-academicjournal": 2, "B-album": 3, "I-album": 4, "B-algorithm": 5, "I-algorithm": 6, "B-astronomicalobject": 7, "I-astronomicalobject": 8, "B-award": 9, "I-award": 10, "B-band": 11, "I-band": 12, "B-book": 13, "I-book": 14, "B-chemicalcompound": 15, "I-chemicalcompound": 16, "B-chemicalelement": 17, "I-chemicalelement": 18, "B-conference": 19, "I-conference": 20, "B-country": 21, "I-country": 22, "B-discipline": 23, "I-discipline": 24, "B-election": 25, "I-election": 26, "B-enzyme": 27, "I-enzyme": 28, "B-event": 29, "I-event": 30, "B-field": 31, "I-field": 32, "B-literarygenre": 33, "I-literarygenre": 34, "B-location": 35, "I-location": 36, "B-magazine": 37, "I-magazine": 38, "B-metrics": 39, "I-metrics": 40, "B-misc": 41, "I-misc": 42, "B-musicalartist": 43, "I-musicalartist": 44, "B-musicalinstrument": 45, "I-musicalinstrument": 46, "B-musicgenre": 47, "I-musicgenre": 48, "B-organisation": 49, "I-organisation": 50, "B-person": 51, "I-person": 52, "B-poem": 53, "I-poem": 54, "B-politicalparty": 55, "I-politicalparty": 56, "B-politician": 57, "I-politician": 58, "B-product": 59, "I-product": 60, "B-programlang": 61, "I-programlang": 62, "B-protein": 63, "I-protein": 64, "B-researcher": 65, "I-researcher": 66, "B-scientist": 67, "I-scientist": 68, "B-song": 69, "I-song": 70, "B-task": 71, "I-task": 72, "B-theory": 73, "I-theory": 74, "B-university": 75, "I-university": 76, "B-writer": 77, "I-writer": 78}
id2tag = {id: tag for tag, id in tag2id.items()}
id2tag

{0: 'O',
 1: 'B-academicjournal',
 2: 'I-academicjournal',
 3: 'B-album',
 4: 'I-album',
 5: 'B-algorithm',
 6: 'I-algorithm',
 7: 'B-astronomicalobject',
 8: 'I-astronomicalobject',
 9: 'B-award',
 10: 'I-award',
 11: 'B-band',
 12: 'I-band',
 13: 'B-book',
 14: 'I-book',
 15: 'B-chemicalcompound',
 16: 'I-chemicalcompound',
 17: 'B-chemicalelement',
 18: 'I-chemicalelement',
 19: 'B-conference',
 20: 'I-conference',
 21: 'B-country',
 22: 'I-country',
 23: 'B-discipline',
 24: 'I-discipline',
 25: 'B-election',
 26: 'I-election',
 27: 'B-enzyme',
 28: 'I-enzyme',
 29: 'B-event',
 30: 'I-event',
 31: 'B-field',
 32: 'I-field',
 33: 'B-literarygenre',
 34: 'I-literarygenre',
 35: 'B-location',
 36: 'I-location',
 37: 'B-magazine',
 38: 'I-magazine',
 39: 'B-metrics',
 40: 'I-metrics',
 41: 'B-misc',
 42: 'I-misc',
 43: 'B-musicalartist',
 44: 'I-musicalartist',
 45: 'B-musicalinstrument',
 46: 'I-musicalinstrument',
 47: 'B-musicgenre',
 48: 'I-musicgenre',
 49: 'B-organisation',
 50: 

Мы видим, что для каждой сущности, например сущность писатель (writer) есть две метки: начало метки ('B-writer') и то, что еще находится в этой метке ('I-writer'). Метка 'O' предсталяет из себя лейбл для всех неименнованных сущностей, то есть тех сущностей, которые нас не интересуют. Пример:

In [21]:
example = pd.DataFrame(train_dataset[0])
example["ners"] = example["ner_tags"].apply(lambda x: id2tag[x])
example[["tokens", "ners", "ner_tags"]].head(10)

,tokens,ners,ner_tags
0,Popular,O,0
1,approaches,O,0
2,of,O,0
3,opinion-based,B-product,59
4,recommender,I-product,60
5,system,I-product,60
6,utilize,O,0
7,various,O,0
8,techniques,O,0
9,including,O,0


Таким образом мы понимаем, что `opinion-based recommender system` - это какой-то продукт

### Пересекаются ли лейблы теги между датасетами?

### Распределение тегов по общему датасету

### Распределение тегов по каждой сабсекции

### Токенизация

### Пересечение вокабуляра между датасетами